In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from code_llama_rag.context_builder import ContextBuilder
from code_llama_rag.code_talker import CodeTalker

In [11]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [4]:
# instantiate code-llama in 4 bit mode
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_compute_dtype=torch.bfloat16,
)
llm_model_id = "codellama/CodeLlama-7b-Instruct-hf"
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_id, 
    device_map="auto",       
    quantization_config=bnb_config,         
).eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# instantiate the context builder
folder = "example-code/"
context_builder = ContextBuilder(folder=folder)

In [7]:
# instantiate the code talker
talker = CodeTalker(llm_model, llm_tokenizer, context_builder)

In [8]:
# define the question
question = "'spell_?book.py': What is the purpose of the function abracadabra?"
# get the answer
answer = talker.answer(question)
print(answer)

number of characters in prompt: 576
number of tokens in prompt: 201
chars per token: 2.8656716417910446
The purpose of the function abracadabra is to reverse the order of spells in the spellbook.



In [10]:
question = "What is the purpose of the function abracadabra?"
answer = talker.answer(question)
print(answer)

number of characters in prompt: 575
number of tokens in prompt: 201
chars per token: 2.8606965174129355
The purpose of the function abracadabra is to reverse the order of spells in the spellbook.



In [12]:
question = "List all classes with their functions in a markdown table"
answer = talker.answer(question)

number of characters in prompt: 1050
number of tokens in prompt: 346
chars per token: 3.0346820809248554


In [14]:
printmd(answer)

| Class | Function |
| --- | --- |
| Spellbook | abracadabra |
| Spellbook | list_spells |
| Kitchen | sousChef |
| Kitchen | list_ingredients |


In [25]:
question = "'^class':List all classes with their functions in a markdown table"
answer = talker.answer(question)
printmd(answer)

number of characters in prompt: 1623
number of tokens in prompt: 516
chars per token: 3.145348837209302


| Class Name | Functions |
| --- | --- |
| Spellbook | abracadabra, list_spells |
| Gadget | overclock, check_status |
| Kitchen | sousChef, list_ingredients |


In [18]:
question = "List all file names"
answer = talker.answer(question)
print(answer)

number of characters in prompt: 158
number of tokens in prompt: 62
chars per token: 2.5483870967741935




In [19]:
question = "'.*.py':List all file names"
answer = talker.answer(question)
print(answer)

number of characters in prompt: 1585
number of tokens in prompt: 509
chars per token: 3.1139489194499017
['example-code/magic/spell_book.py', 'example-code/tech/gadget.py', 'example-code/food/kitchen.py']



In [24]:
question = "'.*.py\\b':List all file names and briefly describe the contents"
answer = talker.answer(question)
printmd(answer)

number of characters in prompt: 1619
number of tokens in prompt: 514
chars per token: 3.149805447470817


The files are:

* `magic/spell_book.py` - A class for a spellbook with a method to reverse the order of spells.
* `tech/gadget.py` - A class for a gadget with a method to overclock its performance level.
* `food/kitchen.py` - A class for a kitchen with a method to prepare ingredients by chopping them finely.
